In [1]:
from citipy import citipy
import numpy as np
import pandas as pd
# Import the requests library.
import requests

# Import the time library.
import time

# Import the datetime module from the datetime library.
from datetime import datetime

# Import the API key.
from config import weather_api_key

In [2]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [3]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)

In [4]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [5]:
# Create a list for holding the cities.
cities = []


# Identify nearest city for each latitude and longitude combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

779

In [6]:
# List of city data
city_data = []

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the url, record, and set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        city_weather = requests.get(city_url).json()
        # Parse out the max temp, humidity, and cloudiness
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather["weather"][0]["description"]       
            
            # Append the City information into city_data list
        city_data.append({"City": city.title(), 
                          "Lat": city_lat, 
                          "Lng": city_lng, 
                          "Max Temp": city_max_temp,
                          "Current Description": city_weather_description,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country})
    # If an error is experienced, skip the city

    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | avarua
Processing Record 2 of Set 1 | jamestown
Processing Record 3 of Set 1 | mar del plata
Processing Record 4 of Set 1 | boa vista
Processing Record 5 of Set 1 | albany
Processing Record 6 of Set 1 | kodiak
Processing Record 7 of Set 1 | yellowknife
Processing Record 8 of Set 1 | yurya
City not found. Skipping...
Processing Record 9 of Set 1 | rikitea
Processing Record 10 of Set 1 | geraldton
Processing Record 11 of Set 1 | abu dhabi
Processing Record 12 of Set 1 | saskylakh
Processing Record 13 of Set 1 | san patricio
Processing Record 14 of Set 1 | eberbach
Processing Record 15 of Set 1 | linhares
Processing Record 16 of Set 1 | novyy nekouz
Processing Record 17 of Set 1 | vardo
Processing Record 18 of Set 1 | seoni malwa
Processing Record 19 of Set 1 | mys shmidta
City not found. Skipping...
Processing Record 20 of Set 1 | zaraza
Processing Record 21 of Set 1 | devonport
Processing Record 2

In [7]:
len(city_data)

714

In [8]:
# Convert array of dictionaries to a pandas DataFrame.
city_data_df = pd.DataFrame(city_data)

In [10]:
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country
0,Avarua,-21.2078,-159.7750,71.65,overcast clouds,78,100,11.50,CK
1,Jamestown,42.0970,-79.2353,77.02,overcast clouds,59,100,9.22,US
2,Mar Del Plata,-38.0023,-57.5575,55.09,overcast clouds,98,100,3.00,AR
3,Boa Vista,2.8197,-60.6733,80.26,overcast clouds,79,99,2.10,BR
4,Albany,42.6001,-73.9662,80.37,overcast clouds,51,99,0.65,US
5,Kodiak,57.7900,-152.4072,52.09,clear sky,72,0,9.89,US
6,Yellowknife,62.4560,-114.3525,62.01,overcast clouds,59,98,3.98,CA
7,Rikitea,-23.1203,-134.9692,74.10,overcast clouds,73,98,12.73,PF
8,Geraldton,-28.7667,114.6000,57.61,overcast clouds,62,100,10.36,AU
9,Abu Dhabi,24.4667,54.3667,91.42,clear sky,67,0,9.22,AE


In [11]:
city_data_df.columns

Index(['City', 'Lat', 'Lng', 'Max Temp', 'Current Description', 'Humidity',
       'Cloudiness', 'Wind Speed', 'Country'],
      dtype='object')

In [12]:
# Reorder the columns in the order you want them to appear.
new_column_order = ["City", "Country", "Lat",
                    "Lng", "Max Temp", "Humidity", 
                    "Cloudiness", "Wind Speed", 
                    "Current Description"] 

# Assign a new or the same DataFrame the new column order.
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Avarua,CK,-21.2078,-159.7750,71.65,78,100,11.50,overcast clouds
1,Jamestown,US,42.0970,-79.2353,77.02,59,100,9.22,overcast clouds
2,Mar Del Plata,AR,-38.0023,-57.5575,55.09,98,100,3.00,overcast clouds
3,Boa Vista,BR,2.8197,-60.6733,80.26,79,99,2.10,overcast clouds
4,Albany,US,42.6001,-73.9662,80.37,51,99,0.65,overcast clouds
5,Kodiak,US,57.7900,-152.4072,52.09,72,0,9.89,clear sky
6,Yellowknife,CA,62.4560,-114.3525,62.01,59,98,3.98,overcast clouds
7,Rikitea,PF,-23.1203,-134.9692,74.10,73,98,12.73,overcast clouds
8,Geraldton,AU,-28.7667,114.6000,57.61,62,100,10.36,overcast clouds
9,Abu Dhabi,AE,24.4667,54.3667,91.42,67,0,9.22,clear sky


In [17]:
# Create the output File (CSV)
output_data_file = "WeatherPy_Database.csv"
# Export the city_data into a csv
city_data_df.to_csv(output_data_file, index_label="City_ID")